In [ ]:
# Instalación de dependencias necesarias
# Ejecutar: pip install -r requirements.txt
# O instalar individualmente:
%pip install flask numpy pandas networkx scikit-learn folium


: 

In [ ]:
"""
Sistema Logístico Agrícola - Yuma County, Arizona

Este notebook muestra cómo usar el sistema de grafo agrícola.
Para la aplicación web interactiva, ejecutar: python app.py
"""

from agricultural_graph import sistema_agricola

# El sistema ya está inicializado automáticamente
print("Sistema agrícola cargado:")
print(f"  - Nodos: {sistema_agricola.G_agricola.number_of_nodes()}")
print(f"  - Aristas: {sistema_agricola.G_agricola.number_of_edges()}")
print(f"  - Parcelas: {len(sistema_agricola.df_parcelas)}")
print(f"  - Centros de acopio: {len(sistema_agricola.df_acopios)}")
print(f"  - Plantas extractoras: {len(sistema_agricola.df_planta)}")

In [ ]:
# Ejemplo: Obtener información de una parcela
parcela_ejemplo = sistema_agricola.df_parcelas.iloc[0]
print("Ejemplo de parcela:")
print(parcela_ejemplo[['id', 'cultivo', 'area_hectareas', 'produccion_estimada_ton']])

# ============================================
# 2. GENERAR CENTROS DE ACOPIO
# ============================================
num_centros_acopio = 5
centros_acopio = []

for i in range(num_centros_acopio):
    lat = np.random.uniform(YUMA_BOUNDS['min_lat'], YUMA_BOUNDS['max_lat'])
    lon = np.random.uniform(YUMA_BOUNDS['min_lon'], YUMA_BOUNDS['max_lon'])
    
    # Capacidad de almacenamiento en toneladas
    capacidad = np.random.uniform(500, 2000)
    
    centros_acopio.append({
        'id': f'ACOPIO_{i+1:02d}',
        'tipo': 'centro_acopio',
        'latitud': lat,
        'longitud': lon,
        'capacidad_ton': capacidad,
        'tiene_cadena_frio': np.random.choice([True, False], p=[0.8, 0.2]),
        'num_camiones_disponibles': np.random.randint(2, 8)
    })

df_acopios = pd.DataFrame(centros_acopio)

# ============================================
# 3. GENERAR PLANTA EXTRACTORA
# ============================================
# Una sola planta extractora, ubicada estratégicamente
planta_extractora = [{
    'id': 'PLANTA_EXTRACTORA_01',
    'tipo': 'planta_extractora',
    'latitud': YUMA_CENTER_LAT + np.random.uniform(-0.1, 0.1),
    'longitud': YUMA_CENTER_LON + np.random.uniform(-0.1, 0.1),
    'capacidad_procesamiento_ton_dia': 5000,
    'horario_operacion': '24/7',
    'requiere_cadena_frio': True
}]

df_planta = pd.DataFrame(planta_extractora)

# ============================================
# 4. COMBINAR TODOS LOS NODOS
# ============================================
# Crear DataFrame unificado de nodos
df_nodos = pd.concat([
    df_parcelas[['id', 'tipo', 'latitud', 'longitud']],
    df_acopios[['id', 'tipo', 'latitud', 'longitud']],
    df_planta[['id', 'tipo', 'latitud', 'longitud']]
], ignore_index=True)

# Agregar atributos completos como diccionario en cada nodo
nodos_completos = []
for idx, row in df_nodos.iterrows():
    nodo_data = row.to_dict()
    
    # Agregar atributos específicos según el tipo
    if row['tipo'] == 'parcela_cultivo':
        parcela_info = df_parcelas[df_parcelas['id'] == row['id']].iloc[0]
        nodo_data.update({
            'cultivo': parcela_info['cultivo'],
            'area_hectareas': parcela_info['area_hectareas'],
            'produccion_estimada_ton': parcela_info['produccion_estimada_ton'],
            'capacidad_almacenamiento_ton': parcela_info['capacidad_almacenamiento_ton'],
            'tiene_cuarto_frio': parcela_info['tiene_cuarto_frio']
        })
    elif row['tipo'] == 'centro_acopio':
        acopio_info = df_acopios[df_acopios['id'] == row['id']].iloc[0]
        nodo_data.update({
            'capacidad_ton': acopio_info['capacidad_ton'],
            'tiene_cadena_frio': acopio_info['tiene_cadena_frio'],
            'num_camiones_disponibles': acopio_info['num_camiones_disponibles']
        })
    elif row['tipo'] == 'planta_extractora':
        planta_info = df_planta[df_planta['id'] == row['id']].iloc[0]
        nodo_data.update({
            'capacidad_procesamiento_ton_dia': planta_info['capacidad_procesamiento_ton_dia'],
            'horario_operacion': planta_info['horario_operacion'],
            'requiere_cadena_frio': planta_info['requiere_cadena_frio']
        })
    
    nodos_completos.append(nodo_data)

df_nodos_completos = pd.DataFrame(nodos_completos)

print("=" * 60)
print("RESUMEN DE NODOS GENERADOS")
print("=" * 60)
print(f"Total de nodos: {len(df_nodos_completos)}")
print(f"  - Parcelas de cultivo: {len(df_parcelas)}")
print(f"  - Centros de acopio: {len(df_acopios)}")
print(f"  - Plantas extractoras: {len(df_planta)}")
print("\nPrimeras parcelas:")
display(df_parcelas.head(3))
print("\nCentros de acopio:")
display(df_acopios)
print("\nPlanta extractora:")
display(df_planta)


In [ ]:
# Ejemplo: Calcular ruta entre dos nodos
nodo1 = sistema_agricola.df_parcelas.iloc[0]['id']
nodo2 = sistema_agricola.df_acopios.iloc[0]['id']

ruta = sistema_agricola.calcular_ruta_entre_nodos(nodo1, nodo2)
if ruta:
    print(f"Ruta entre {nodo1} y {nodo2}:")
    print(f"  Distancia: {ruta['distancia_km']:.2f} km")
    print(f"  Tiempo: {ruta['tiempo_minutos']:.1f} min")
    print(f"  Costo: ${ruta['costo_por_ton']:.2f}/ton")
else:
    print("No hay ruta disponible")

# ============================================
# GENERAR ARISTAS (CONEXIONES)
# ============================================
# Estrategia de conexión:
# 1. Parcelas -> Centros de acopio (cada parcela conecta a los 2-3 acopios más cercanos)
# 2. Centros de acopio -> Planta extractora (todos los acopios conectan a la planta)
# 3. Algunas parcelas -> Planta extractora directamente (para grandes productores)

aristas = []

# 1. CONEXIONES: Parcelas -> Centros de Acopio
for _, parcela in df_parcelas.iterrows():
    # Calcular distancias a todos los centros de acopio
    distancias_acopios = []
    for _, acopio in df_acopios.iterrows():
        distancia = calcular_distancia_haversine(
            parcela['latitud'], parcela['longitud'],
            acopio['latitud'], acopio['longitud']
        )
        distancias_acopios.append((acopio['id'], distancia))
    
    # Ordenar por distancia y tomar los 2-3 más cercanos
    distancias_acopios.sort(key=lambda x: x[1])
    num_conexiones = np.random.randint(2, min(4, len(distancias_acopios) + 1))
    
    for acopio_id, distancia in distancias_acopios[:num_conexiones]:
        # Determinar tipo de camino basado en distancia y ubicación
        if distancia < 5000:  # Menos de 5km
            tipo_camino = np.random.choice(['pavimentado', 'grava'], p=[0.7, 0.3])
            velocidad_promedio = np.random.uniform(40, 60)  # km/h
        elif distancia < 15000:  # Entre 5-15km
            tipo_camino = np.random.choice(['pavimentado', 'grava', 'tierra'], p=[0.5, 0.3, 0.2])
            velocidad_promedio = np.random.uniform(35, 55)
        else:  # Más de 15km
            tipo_camino = np.random.choice(['pavimentado', 'grava', 'tierra'], p=[0.4, 0.4, 0.2])
            velocidad_promedio = np.random.uniform(30, 50)
        
        # Calcular tiempo de viaje (en segundos)
        tiempo_segundos = (distancia / 1000) / velocidad_promedio * 3600
        
        # Calcular costo (dólares por tonelada)
        # Factores: distancia, tipo de camino, combustible
        costo_combustible_por_km = 0.15  # $0.15 por km
        costo_base = (distancia / 1000) * costo_combustible_por_km
        
        # Penalización por tipo de camino
        if tipo_camino == 'tierra':
            costo_base *= 1.3
        elif tipo_camino == 'grava':
            costo_base *= 1.1
        
        # Costo por tonelada transportada
        costo_por_ton = costo_base
        
        # Accesibilidad en temporada de lluvias (0-1, donde 1 es totalmente accesible)
        if tipo_camino == 'pavimentado':
            accesibilidad_lluvia = np.random.uniform(0.85, 1.0)
        elif tipo_camino == 'grava':
            accesibilidad_lluvia = np.random.uniform(0.5, 0.85)
        else:  # tierra
            accesibilidad_lluvia = np.random.uniform(0.2, 0.6)
        
        aristas.append({
            'origen': parcela['id'],
            'destino': acopio_id,
            'distancia_metros': distancia,
            'distancia_km': distancia / 1000,
            'tiempo_segundos': tiempo_segundos,
            'tiempo_minutos': tiempo_segundos / 60,
            'costo_por_ton_dolares': costo_por_ton,
            'tipo_camino': tipo_camino,
            'velocidad_promedio_kmh': velocidad_promedio,
            'accesibilidad_lluvia': accesibilidad_lluvia,
            'tipo_conexion': 'parcela_acopio'
        })

# 2. CONEXIONES: Centros de Acopio -> Planta Extractora
planta_id = df_planta.iloc[0]['id']
planta_lat = df_planta.iloc[0]['latitud']
planta_lon = df_planta.iloc[0]['longitud']

for _, acopio in df_acopios.iterrows():
    distancia = calcular_distancia_haversine(
        acopio['latitud'], acopio['longitud'],
        planta_lat, planta_lon
    )
    
    # Conexiones a planta suelen ser por carreteras principales
    tipo_camino = np.random.choice(['pavimentado', 'grava'], p=[0.9, 0.1])
    velocidad_promedio = np.random.uniform(50, 70)  # Carreteras principales más rápidas
    
    tiempo_segundos = (distancia / 1000) / velocidad_promedio * 3600
    
    # Costo más bajo por ser ruta principal
    costo_combustible_por_km = 0.12
    costo_por_ton = (distancia / 1000) * costo_combustible_por_km
    
    # Mejor accesibilidad en carreteras principales
    accesibilidad_lluvia = np.random.uniform(0.9, 1.0)
    
    aristas.append({
        'origen': acopio['id'],
        'destino': planta_id,
        'distancia_metros': distancia,
        'distancia_km': distancia / 1000,
        'tiempo_segundos': tiempo_segundos,
        'tiempo_minutos': tiempo_segundos / 60,
        'costo_por_ton_dolares': costo_por_ton,
        'tipo_camino': tipo_camino,
        'velocidad_promedio_kmh': velocidad_promedio,
        'accesibilidad_lluvia': accesibilidad_lluvia,
        'tipo_conexion': 'acopio_planta'
    })

# 3. CONEXIONES DIRECTAS: Parcelas grandes -> Planta Extractora (solo algunas)
parcelas_grandes = df_parcelas[df_parcelas['area_hectareas'] > 100]
num_directas = min(5, len(parcelas_grandes))

for _, parcela in parcelas_grandes.sample(n=num_directas, random_state=42).iterrows():
    distancia = calcular_distancia_haversine(
        parcela['latitud'], parcela['longitud'],
        planta_lat, planta_lon
    )
    
    # Rutas directas suelen ser por carreteras principales
    tipo_camino = 'pavimentado'
    velocidad_promedio = np.random.uniform(55, 70)
    
    tiempo_segundos = (distancia / 1000) / velocidad_promedio * 3600
    costo_por_ton = (distancia / 1000) * 0.12
    accesibilidad_lluvia = np.random.uniform(0.9, 1.0)
    
    aristas.append({
        'origen': parcela['id'],
        'destino': planta_id,
        'distancia_metros': distancia,
        'distancia_km': distancia / 1000,
        'tiempo_segundos': tiempo_segundos,
        'tiempo_minutos': tiempo_segundos / 60,
        'costo_por_ton_dolares': costo_por_ton,
        'tipo_camino': tipo_camino,
        'velocidad_promedio_kmh': velocidad_promedio,
        'accesibilidad_lluvia': accesibilidad_lluvia,
        'tipo_conexion': 'parcela_planta_directa'
    })

# Crear DataFrame de aristas
df_aristas = pd.DataFrame(aristas)

# Agregar aristas al grafo
for _, edge in df_aristas.iterrows():
    G_agricola.add_edge(
        edge['origen'],
        edge['destino'],
        distancia_metros=edge['distancia_metros'],
        distancia_km=edge['distancia_km'],
        tiempo_segundos=edge['tiempo_segundos'],
        tiempo_minutos=edge['tiempo_minutos'],
        costo_por_ton_dolares=edge['costo_por_ton_dolares'],
        tipo_camino=edge['tipo_camino'],
        velocidad_promedio_kmh=edge['velocidad_promedio_kmh'],
        accesibilidad_lluvia=edge['accesibilidad_lluvia'],
        tipo_conexion=edge['tipo_conexion']
    )

print("=" * 60)
print("RESUMEN DEL GRAFO AGRÍCOLA")
print("=" * 60)
print(f"Total de nodos: {G_agricola.number_of_nodes()}")
print(f"Total de aristas: {G_agricola.number_of_edges()}")
print(f"\nTipos de conexiones:")
print(df_aristas['tipo_conexion'].value_counts())
print(f"\nTipos de caminos:")
print(df_aristas['tipo_camino'].value_counts())
print("\nEstadísticas de las aristas:")
print(df_aristas[['distancia_km', 'tiempo_minutos', 'costo_por_ton_dolares', 'accesibilidad_lluvia']].describe())
print("\nPrimeras aristas:")
display(df_aristas.head(5))


In [ ]:
import folium
from folium import plugins

# Crear mapa centrado en Yuma County
mapa_yuma = folium.Map(
    location=[YUMA_CENTER_LAT, YUMA_CENTER_LON],
    zoom_start=10,
    tiles='OpenStreetMap'
)

# Definir colores según tipo de nodo
color_map = {
    'parcela_cultivo': 'green',
    'centro_acopio': 'blue',
    'planta_extractora': 'red'
}

# Agregar marcadores para cada nodo
for _, nodo in df_nodos_completos.iterrows():
    color = color_map.get(nodo['tipo'], 'gray')
    
    # Crear popup con información del nodo
    if nodo['tipo'] == 'parcela_cultivo':
        popup_text = f"""
        <b>{nodo['id']}</b><br>
        Tipo: Parcela de Cultivo<br>
        Cultivo: {nodo.get('cultivo', 'N/A')}<br>
        Área: {nodo.get('area_hectareas', 0):.1f} ha<br>
        Producción estimada: {nodo.get('produccion_estimada_ton', 0):.1f} ton
        """
        icon = 'leaf'
    elif nodo['tipo'] == 'centro_acopio':
        popup_text = f"""
        <b>{nodo['id']}</b><br>
        Tipo: Centro de Acopio<br>
        Capacidad: {nodo.get('capacidad_ton', 0):.1f} ton<br>
        Camiones disponibles: {nodo.get('num_camiones_disponibles', 0)}
        """
        icon = 'warehouse'
    else:  # planta_extractora
        popup_text = f"""
        <b>{nodo['id']}</b><br>
        Tipo: Planta Extractora<br>
        Capacidad: {nodo.get('capacidad_procesamiento_ton_dia', 0):.0f} ton/día
        """
        icon = 'industry'
    
    folium.Marker(
        location=[nodo['latitud'], nodo['longitud']],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color=color, icon=icon, prefix='fa'),
        tooltip=nodo['id']
    ).add_to(mapa_yuma)

# Agregar líneas para las aristas (con colores según tipo de camino)
for _, arista in df_aristas.iterrows():
    origen = df_nodos_completos[df_nodos_completos['id'] == arista['origen']].iloc[0]
    destino = df_nodos_completos[df_nodos_completos['id'] == arista['destino']].iloc[0]
    
    # Color según tipo de camino
    if arista['tipo_camino'] == 'pavimentado':
        color_linea = 'green'
        opacity = 0.6
    elif arista['tipo_camino'] == 'grava':
        color_linea = 'orange'
        opacity = 0.5
    else:  # tierra
        color_linea = 'brown'
        opacity = 0.4
    
    # Grosor de línea según accesibilidad en lluvia
    weight = 2 + (arista['accesibilidad_lluvia'] * 3)
    
    popup_linea = f"""
    <b>Ruta: {arista['origen']} → {arista['destino']}</b><br>
    Distancia: {arista['distancia_km']:.2f} km<br>
    Tiempo: {arista['tiempo_minutos']:.1f} min<br>
    Costo: ${arista['costo_por_ton_dolares']:.2f}/ton<br>
    Tipo: {arista['tipo_camino']}<br>
    Accesibilidad lluvia: {arista['accesibilidad_lluvia']:.2%}
    """
    
    folium.PolyLine(
        locations=[[origen['latitud'], origen['longitud']],
                   [destino['latitud'], destino['longitud']]],
        color=color_linea,
        weight=weight,
        opacity=opacity,
        popup=folium.Popup(popup_linea, max_width=300),
        tooltip=f"{arista['distancia_km']:.1f} km - {arista['tipo_camino']}"
    ).add_to(mapa_yuma)

# Agregar leyenda
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; right: 50px; width: 200px; height: 150px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     font-size:14px; padding: 10px">
     <h4>Leyenda</h4>
     <p><i class="fa fa-leaf" style="color:green"></i> Parcelas</p>
     <p><i class="fa fa-warehouse" style="color:blue"></i> Centros Acopio</p>
     <p><i class="fa fa-industry" style="color:red"></i> Planta Extractora</p>
     <p><span style="color:green">━━━</span> Pavimentado</p>
     <p><span style="color:orange">━━━</span> Grava</p>
     <p><span style="color:brown">━━━</span> Tierra</p>
</div>
'''
mapa_yuma.get_root().html.add_child(folium.Element(legend_html))

print("Mapa generado. Mostrando grafo agrícola de Yuma County:")
mapa_yuma


In [ ]:
# ============================================
# PREPARACIÓN PARA INTEGRACIÓN DE IA
# ============================================
# Esta sección prepara la estructura de datos para el sistema de predicción de producción
# y optimización de rutas basado en IA

# 1. Crear dataset de características para predicción de producción
def crear_dataset_produccion():
    """
    Crea un dataset con características que pueden usarse para predecir producción
    de las parcelas usando modelos de IA.
    """
    features_produccion = []
    
    for _, parcela in df_parcelas.iterrows():
        # Características de la parcela
        features = {
            'parcela_id': parcela['id'],
            'cultivo': parcela['cultivo'],
            'area_hectareas': parcela['area_hectareas'],
            'produccion_estimada_ton': parcela['produccion_estimada_ton'],
            'tiene_cuarto_frio': 1 if parcela['tiene_cuarto_frio'] else 0,
            
            # Características logísticas (conectividad)
            'num_rutas_disponibles': len([a for a in df_aristas.itertuples() 
                                          if a.origen == parcela['id']]),
            'distancia_promedio_acopios': df_aristas[
                (df_aristas['origen'] == parcela['id']) & 
                (df_aristas['tipo_conexion'] == 'parcela_acopio')
            ]['distancia_km'].mean() if len(df_aristas[
                (df_aristas['origen'] == parcela['id']) & 
                (df_aristas['tipo_conexion'] == 'parcela_acopio')
            ]) > 0 else 0,
            
            'accesibilidad_promedio_lluvia': df_aristas[
                df_aristas['origen'] == parcela['id']
            ]['accesibilidad_lluvia'].mean() if len(df_aristas[
                df_aristas['origen'] == parcela['id']
            ]) > 0 else 0,
            
            'costo_promedio_transporte': df_aristas[
                df_aristas['origen'] == parcela['id']
            ]['costo_por_ton_dolares'].mean() if len(df_aristas[
                df_aristas['origen'] == parcela['id']
            ]) > 0 else 0,
        }
        
        features_produccion.append(features)
    
    return pd.DataFrame(features_produccion)

df_features_produccion = crear_dataset_produccion()

# 2. Función para actualizar producción predicha en el grafo
def actualizar_produccion_predicha(parcela_id, produccion_predicha):
    """
    Actualiza la producción predicha de una parcela en el grafo.
    
    Parámetros:
    -----------
    parcela_id : str
        ID de la parcela
    produccion_predicha : float
        Producción predicha en toneladas
    """
    if parcela_id in G_agricola.nodes():
        G_agricola.nodes[parcela_id]['produccion_predicha_ton'] = produccion_predicha
        G_agricola.nodes[parcela_id]['produccion_actualizada'] = True
        return True
    return False

# 3. Función para calcular rutas óptimas basadas en producción predicha
def calcular_rutas_optimas_por_produccion(parcela_id, produccion_predicha, 
                                          criterio='costo', 
                                          considerar_lluvia=False):
    """
    Calcula las rutas óptimas desde una parcela considerando la producción predicha.
    
    Parámetros:
    -----------
    parcela_id : str
        ID de la parcela
    produccion_predicha : float
        Producción predicha en toneladas
    criterio : str
        'costo', 'tiempo', 'distancia', o 'accesibilidad'
    considerar_lluvia : bool
        Si True, ajusta los pesos según accesibilidad en lluvia
    """
    if parcela_id not in G_agricola.nodes():
        return None
    
    rutas_disponibles = []
    
    # Obtener todas las aristas salientes de la parcela
    for destino in G_agricola.successors(parcela_id):
        edge_data = G_agricola[parcela_id][destino]
        
        # Calcular peso según criterio
        if criterio == 'costo':
            peso = edge_data['costo_por_ton_dolares'] * produccion_predicha
        elif criterio == 'tiempo':
            peso = edge_data['tiempo_minutos']
        elif criterio == 'distancia':
            peso = edge_data['distancia_km']
        elif criterio == 'accesibilidad':
            peso = 1 / edge_data['accesibilidad_lluvia']  # Invertir para minimizar
        else:
            peso = edge_data['costo_por_ton_dolares']
        
        # Ajustar por accesibilidad en lluvia si se solicita
        if considerar_lluvia:
            peso = peso / edge_data['accesibilidad_lluvia']
        
        rutas_disponibles.append({
            'destino': destino,
            'peso': peso,
            'distancia_km': edge_data['distancia_km'],
            'tiempo_minutos': edge_data['tiempo_minutos'],
            'costo_total': edge_data['costo_por_ton_dolares'] * produccion_predicha,
            'accesibilidad_lluvia': edge_data['accesibilidad_lluvia'],
            'tipo_camino': edge_data['tipo_camino']
        })
    
    # Ordenar por peso (menor es mejor)
    rutas_disponibles.sort(key=lambda x: x['peso'])
    
    return rutas_disponibles

# 4. Función para actualizar pesos de aristas basado en predicciones
def actualizar_pesos_aristas_por_produccion():
    """
    Actualiza los pesos de las aristas considerando las producciones predichas.
    Esto permite ajustar dinámicamente las rutas según predicciones de IA.
    """
    for u, v, data in G_agricola.edges(data=True):
        # Si el origen es una parcela, actualizar costo total basado en producción predicha
        if G_agricola.nodes[u].get('tipo') == 'parcela_cultivo':
            produccion = G_agricola.nodes[u].get('produccion_predicha_ton', 
                                                G_agricola.nodes[u].get('produccion_estimada_ton', 0))
            
            # Actualizar costo total estimado
            data['costo_total_estimado'] = data['costo_por_ton_dolares'] * produccion
            
            # Actualizar tiempo total estimado (considerando carga/descarga)
            tiempo_carga = produccion * 0.5  # 0.5 min por tonelada
            data['tiempo_total_estimado_min'] = data['tiempo_minutos'] + tiempo_carga

# 5. Crear dataset para entrenamiento de modelo de predicción
def preparar_dataset_entrenamiento():
    """
    Prepara un dataset con características que pueden usarse para entrenar
    un modelo de predicción de producción.
    
    NOTA: En producción real, esto incluiría datos históricos de:
    - Producción pasada
    - Condiciones climáticas
    - Calidad del suelo
    - Uso de fertilizantes
    - Etc.
    """
    # Por ahora, usamos los datos simulados como base
    # En producción, se agregarían más características
    dataset = df_features_produccion.copy()
    
    # Agregar características adicionales que podrían venir de sensores/satélites
    dataset['indice_vegetacion_simulado'] = np.random.uniform(0.3, 0.9, len(dataset))
    dataset['humedad_suelo_simulado'] = np.random.uniform(20, 60, len(dataset))
    dataset['temperatura_promedio'] = np.random.uniform(25, 35, len(dataset))
    
    return dataset

df_entrenamiento = preparar_dataset_entrenamiento()

print("=" * 60)
print("ESTRUCTURA PREPARADA PARA INTEGRACIÓN DE IA")
print("=" * 60)
print(f"\nDataset de características para predicción:")
print(f"  - Filas: {len(df_features_produccion)}")
print(f"  - Columnas: {len(df_features_produccion.columns)}")
print(f"\nColumnas disponibles:")
print(df_features_produccion.columns.tolist())
print("\nEjemplo de dataset de entrenamiento:")
display(df_entrenamiento.head(3))
print("\nFunciones disponibles:")
print("  - actualizar_produccion_predicha(parcela_id, produccion)")
print("  - calcular_rutas_optimas_por_produccion(parcela_id, produccion, criterio)")
print("  - actualizar_pesos_aristas_por_produccion()")
print("\n" + "=" * 60)
print("PRÓXIMOS PASOS PARA INTEGRACIÓN DE IA:")
print("=" * 60)
print("1. Recopilar datos históricos de producción de parcelas")
print("2. Integrar datos de sensores (clima, suelo, satélites)")
print("3. Entrenar modelo de predicción (Random Forest, XGBoost, Neural Network)")
print("4. Implementar actualización periódica de predicciones")
print("5. Usar predicciones para optimizar rutas en tiempo real")
print("6. Considerar restricciones (capacidad de acopios, disponibilidad de camiones)")


In [ ]:
# ============================================
# EJEMPLO DE USO: SIMULACIÓN DE PREDICCIÓN DE IA
# ============================================
# Esta celda muestra cómo se integraría un modelo de IA para predecir producción
# y ajustar las rutas de recolección

# Ejemplo 1: Simular predicción de producción para una parcela
parcela_ejemplo = df_parcelas.iloc[0]['id']
produccion_original = df_parcelas.iloc[0]['produccion_estimada_ton']

# Simular que un modelo de IA predice una producción diferente
# (En producción real, esto vendría de un modelo entrenado)
produccion_predicha_ia = produccion_original * np.random.uniform(0.8, 1.2)

print("=" * 60)
print("EJEMPLO: ACTUALIZACIÓN CON PREDICCIÓN DE IA")
print("=" * 60)
print(f"\nParcela: {parcela_ejemplo}")
print(f"Producción original estimada: {produccion_original:.2f} ton")
print(f"Producción predicha por IA: {produccion_predicha_ia:.2f} ton")
print(f"Diferencia: {(produccion_predicha_ia - produccion_original):.2f} ton ({((produccion_predicha_ia/produccion_original - 1)*100):.1f}%)")

# Actualizar producción en el grafo
actualizar_produccion_predicha(parcela_ejemplo, produccion_predicha_ia)

# Ejemplo 2: Calcular rutas óptimas considerando la producción predicha
print("\n" + "=" * 60)
print("RUTAS ÓPTIMAS CONSIDERANDO PRODUCCIÓN PREDICHA")
print("=" * 60)

# Por costo
rutas_costo = calcular_rutas_optimas_por_produccion(
    parcela_ejemplo, 
    produccion_predicha_ia, 
    criterio='costo'
)

print("\nTop 3 rutas por COSTO:")
for i, ruta in enumerate(rutas_costo[:3], 1):
    print(f"\n{i}. Destino: {ruta['destino']}")
    print(f"   Costo total: ${ruta['costo_total']:.2f}")
    print(f"   Distancia: {ruta['distancia_km']:.2f} km")
    print(f"   Tiempo: {ruta['tiempo_minutos']:.1f} min")
    print(f"   Tipo camino: {ruta['tipo_camino']}")
    print(f"   Accesibilidad lluvia: {ruta['accesibilidad_lluvia']:.2%}")

# Por tiempo
rutas_tiempo = calcular_rutas_optimas_por_produccion(
    parcela_ejemplo, 
    produccion_predicha_ia, 
    criterio='tiempo'
)

print("\nTop 3 rutas por TIEMPO:")
for i, ruta in enumerate(rutas_tiempo[:3], 1):
    print(f"\n{i}. Destino: {ruta['destino']}")
    print(f"   Tiempo: {ruta['tiempo_minutos']:.1f} min")
    print(f"   Costo total: ${ruta['costo_total']:.2f}")
    print(f"   Distancia: {ruta['distancia_km']:.2f} km")

# Considerando temporada de lluvias
rutas_lluvia = calcular_rutas_optimas_por_produccion(
    parcela_ejemplo, 
    produccion_predicha_ia, 
    criterio='accesibilidad',
    considerar_lluvia=True
)

print("\nTop 3 rutas considerando TEMPORADA DE LLUVIAS:")
for i, ruta in enumerate(rutas_lluvia[:3], 1):
    print(f"\n{i}. Destino: {ruta['destino']}")
    print(f"   Accesibilidad lluvia: {ruta['accesibilidad_lluvia']:.2%}")
    print(f"   Tipo camino: {ruta['tipo_camino']}")
    print(f"   Tiempo: {ruta['tiempo_minutos']:.1f} min")
    print(f"   Costo total: ${ruta['costo_total']:.2f}")

# Ejemplo 3: Actualizar todos los pesos del grafo
print("\n" + "=" * 60)
print("ACTUALIZANDO PESOS DEL GRAFO CON PREDICCIONES")
print("=" * 60)

# Simular predicciones para todas las parcelas
for _, parcela in df_parcelas.iterrows():
    produccion_original = parcela['produccion_estimada_ton']
    # Simular variación de ±20% en predicción
    produccion_predicha = produccion_original * np.random.uniform(0.8, 1.2)
    actualizar_produccion_predicha(parcela['id'], produccion_predicha)

# Actualizar pesos de aristas
actualizar_pesos_aristas_por_produccion()

print("✓ Producciones actualizadas para todas las parcelas")
print("✓ Pesos de aristas recalculados")

# Mostrar ejemplo de arista actualizada
ejemplo_arista = list(G_agricola.edges(data=True))[0]
print(f"\nEjemplo de arista actualizada:")
print(f"  Origen: {ejemplo_arista[0]}")
print(f"  Destino: {ejemplo_arista[1]}")
if 'costo_total_estimado' in ejemplo_arista[2]:
    print(f"  Costo total estimado: ${ejemplo_arista[2]['costo_total_estimado']:.2f}")
if 'tiempo_total_estimado_min' in ejemplo_arista[2]:
    print(f"  Tiempo total estimado: {ejemplo_arista[2]['tiempo_total_estimado_min']:.1f} min")

print("\n" + "=" * 60)
print("ESTRUCTURA LISTA PARA INTEGRACIÓN COMPLETA DE IA")
print("=" * 60)
print("\nEl grafo ahora contiene:")
print("  - Nodos con producción predicha")
print("  - Aristas con pesos actualizados según predicciones")
print("  - Funciones para calcular rutas óptimas dinámicamente")
print("\nPróximo paso: Integrar modelo de ML real para predicciones")


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Seleccionamos variables numéricas (copia del dataset original, buena practica)
df = edges[["length"]].copy()
# Ajuste de valores de velocidad maxima (correcion de error con 50km/h)
df["maxspeed"] = pd.to_numeric(edges["maxspeed"], errors="coerce").fillna(50)
df["travel_time"] = df["length"] / (df["maxspeed"] * 1000 / 3600)

# Normalizamos para despues trabajar con modelos
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

print(df_scaled.head(6))

In [ ]:
import networkx as nx
import numpy as np

# Crear grafo aleatorio geometrico
G_sim = nx.random_geometric_graph(10, radius=0.5, seed=42)

# Añadir pesos simulados
for u, v in G_sim.edges():
    distancia = np.random.uniform(500, 5000)  # en metros
    velocidad = np.random.choice([30, 50, 80])  # km/h
    tipo = np.random.choice(["pavimentado", "tierra", "grava"])
    tiempo = distancia / (velocidad * 1000 / 3600)  # convertir a segundos
    G_sim.edges[u, v]["length"] = distancia
    G_sim.edges[u, v]["maxspeed"] = velocidad
    G_sim.edges[u, v]["tipo"] = tipo
    G_sim.edges[u, v]["travel_time"] = tiempo

# List comprehension para crear tuplas y un nuevo dataset con los valores aleatorios generados
edges_sim = pd.DataFrame(
    [
        (u, v, d["length"], d["maxspeed"], d["tipo"], d["travel_time"])
        for u, v, d in G_sim.edges(data=True)
    ],
    columns=["u", "v", "length", "maxspeed", "tipo", "travel_time"],
)

print(edges_sim.head())

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

# obtener atributos de los nodos
pos = nx.get_node_attributes(G_sim, "pos")
display(pos)
colors = [d["maxspeed"] for _, _, d in G_sim.edges(data=True)]
node_colors = [G_sim.degree(n) for n in G_sim.nodes()]

# Crear etiquetas de aristas con informacion como velocidad de la ruta y tipo
edge_labels = {
    (u, v): f'{d["tipo"]}\n{d["maxspeed"]}km/h' for u, v, d in G_sim.edges(data=True)
}

# Crear etiquetas de los nodos con informacion de los grados
node_labels = {
    node: f"Node {node}\nDegree: {G_sim.degree(node)}" for node in G_sim.nodes()
}

# Dibujar el grafo simulado
plt.figure(figsize=(12, 8))
nx.draw(
    G_sim,
    pos,
    node_color=node_colors,
    edge_color=colors,
    edge_cmap=plt.cm.viridis,
    node_size=1000,
    cmap=plt.cm.Blues,
    with_labels=False,
)

# Añadir etiquetas de las aristas
nx.draw_networkx_edge_labels(G_sim, pos, edge_labels, font_size=8)

# Añadir etiquetas de los nodos
nx.draw_networkx_labels(G_sim, pos, node_labels, font_size=8)

plt.title("Red vial simulada con informacion de nodos y aristas")
plt.show()

In [ ]:
import folium

# Convertir el grafo a GeoDataFrame de aristas
edges_gdf = ox.graph_to_gdfs(G, nodes=False, edges=True)

# Calcular el centroide de todas las aristas (para centrar el mapa)
centroid = edges_gdf.unary_union.centroid
map_center = [centroid.y, centroid.x]

# Crear el mapa centrado
m = folium.Map(location=map_center, zoom_start=12)

# Añadir las aristas al mapa (GeoJSON actual)
folium.GeoJson(data=edges_gdf.__geo_interface__).add_to(m)

# Mostrar el mapa
m